In [230]:
import pandas as pd
import json
import os
import re
from glob import glob
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions

In [231]:
def get_csv_path(filename):
    with open(filename, "r") as line:
        json_content = line.read()
        attributes = json.loads(json_content)
        TWEETS_STORAGE_PATH = attributes["TWEETS_STORAGE_PATH"]
        RESULTS_STORAGE_PATH = attributes["RESULTS_STORAGE_PATH"]
    return TWEETS_STORAGE_PATH, RESULTS_STORAGE_PATH

In [232]:
def get_dataframe(path_csv):
    dataframes = []
    en_lectura = ""
    archivos = [os.path.basename(x) for x in glob(path_csv+'/*.csv')]
    try:
        for arch in archivos:
            en_estudio = arch
            dataframes.append(pd.read_csv(path_csv+'/'+arch))
    except:
        print("Error leyendo el archivo " + en_lectura)
    return pd.concat(dataframes, sort=False)

In [234]:
def remove_accent(s):
  replacements = (
      ("á", "a"),
      ("é", "e"),
      ("í", "i"),
      ("ó", "o"),
      ("ú", "u"),
      ("Á", "A"),
      ("É", "E"),
      ("Í", "I"),
      ("Ó", "O"),
      ("Ú", "U"),
      ("à", "a"),
      ("è", "e"),
      ("ì", "i"),
      ("ò", "o"),
      ("ù", "u"),
      ("À", "A"),
      ("È", "E"),
      ("Ì", "I"),
      ("Ò", "O"),
      ("Ù", "U"),
  )
  for a, b in replacements:
      s = s.replace(a, b)
  return s

In [233]:
def analisis_watson(text):
    try:
      response = nlu.analyze(text=text,features=Features( sentiment=SentimentOptions()), language='es')
    except:
      return 'error'
    return response.result['sentiment']['document']['label']

In [235]:
# defino la instancia de Análisis de lenguaje 
nlu = NaturalLanguageUnderstandingV1(
    version='2018-11-16',
    iam_apikey='NuhsNVWkHHIc27U-Fs5fL-guMFD0aEmAo3qwzxJlMbNL',
    url='https://gateway.watsonplatform.net/natural-language-understanding/api')

In [236]:
tweets_csv_path, results_csv_path = get_csv_path("./Config/config.json")

In [237]:
df = get_dataframe(tweets_csv_path)
print("Dimensión Dataframe: " + str(df.shape))
print("Data Types:\n" + str(df.dtypes))

Dimensión Dataframe: (1800, 9)
Data Types:
Unnamed: 0         int64 
id                 int64 
created_at         object
name               object
followers_count    int64 
full_text          object
favorite_count     int64 
retweet_count      int64 
flag               object
dtype: object


In [238]:
#df['date_time'] = pd.to_datetime(df['created_at'], format= '%Y-%m-%d %H:%M:%S')

In [240]:
pd.set_option('display.max_colwidth', 30)
df.head(5)

,Unnamed: 0,id,created_at,name,followers_count,full_text,favorite_count,retweet_count,flag
0,0,1113517000167448581,2019-04-03 19:01:55,Dgaceyguillo,101,RT @minutounocom: La joven...,0,13,Kirchner
1,1,1113516983679619073,2019-04-03 19:01:51,Yolanda22362688,307,RT @krauseirishotm1: Romin...,0,49,Kirchner
2,2,1113516955158351878,2019-04-03 19:01:44,BramanteLidia,409,RT @PoliticaClarin: Rechaz...,0,1,Kirchner
3,3,1113516898438848513,2019-04-03 19:01:31,migueldelpalmar,112,"RT @PissanoV: A ver gente,...",0,1127,Kirchner
4,4,1113516858513268737,2019-04-03 19:01:21,Dgaceyguillo,101,RT @arilijalad: “Acá hay u...,0,73,Kirchner


In [241]:
df.drop(columns="Unnamed: 0", inplace=True)

In [242]:
pd.set_option('display.max_colwidth', 30)
df.head(2)

,id,created_at,name,followers_count,full_text,favorite_count,retweet_count,flag
0,1113517000167448581,2019-04-03 19:01:55,Dgaceyguillo,101,RT @minutounocom: La joven...,0,13,Kirchner
1,1113516983679619073,2019-04-03 19:01:51,Yolanda22362688,307,RT @krauseirishotm1: Romin...,0,49,Kirchner


In [243]:
df['full_text_no_url'] = df['full_text'].str.replace('http\S+|www.\S+', '', case=False)

In [244]:
pd.set_option('display.max_colwidth', -1)
df.head(5)

,id,created_at,name,followers_count,full_text,favorite_count,retweet_count,flag,full_text_no_url
0,1113517000167448581,2019-04-03 19:01:55,Dgaceyguillo,101,RT @minutounocom: La joven debíaregresar este jueves a la Argentina tras someterse a un tratamiento médico https://t.co/kWL1dwqCKT,0,13,Kirchner,RT @minutounocom: La joven debíaregresar este jueves a la Argentina tras someterse a un tratamiento médico
1,1113516983679619073,2019-04-03 19:01:51,Yolanda22362688,307,"RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner esté acá..."" La verdad que dan asco todos!! Mariano Cúneo…",0,49,Kirchner,"RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner esté acá..."" La verdad que dan asco todos!! Mariano Cúneo…"
2,1113516955158351878,2019-04-03 19:01:44,BramanteLidia,409,RT @PoliticaClarin: Rechazan la unificación de dos causas que podría retrasar el primer juicio oral a Cristina Kirchner https://t.co/HJnQSE…,0,1,Kirchner,RT @PoliticaClarin: Rechazan la unificación de dos causas que podría retrasar el primer juicio oral a Cristina Kirchner
3,1113516898438848513,2019-04-03 19:01:31,migueldelpalmar,112,"RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…",0,1127,Kirchner,"RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…"
4,1113516858513268737,2019-04-03 19:01:21,Dgaceyguillo,101,"RT @arilijalad: “Acá hay una forma de apriete. Te piden tirar mierda para arriba. Declará como arrepentido y te vas a tu casa mañana”, cont…",0,73,Kirchner,"RT @arilijalad: “Acá hay una forma de apriete. Te piden tirar mierda para arriba. Declará como arrepentido y te vas a tu casa mañana”, cont…"


In [245]:
df['full_text_no_url_no_acc'] = df['full_text_no_url'].apply(lambda x: remove_accent(x))

In [246]:
df.head(5)

,id,created_at,name,followers_count,full_text,favorite_count,retweet_count,flag,full_text_no_url,full_text_no_url_no_acc
0,1113517000167448581,2019-04-03 19:01:55,Dgaceyguillo,101,RT @minutounocom: La joven debíaregresar este jueves a la Argentina tras someterse a un tratamiento médico https://t.co/kWL1dwqCKT,0,13,Kirchner,RT @minutounocom: La joven debíaregresar este jueves a la Argentina tras someterse a un tratamiento médico,RT @minutounocom: La joven debiaregresar este jueves a la Argentina tras someterse a un tratamiento medico
1,1113516983679619073,2019-04-03 19:01:51,Yolanda22362688,307,"RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner esté acá..."" La verdad que dan asco todos!! Mariano Cúneo…",0,49,Kirchner,"RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner esté acá..."" La verdad que dan asco todos!! Mariano Cúneo…","RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner este aca..."" La verdad que dan asco todos!! Mariano Cuneo…"
2,1113516955158351878,2019-04-03 19:01:44,BramanteLidia,409,RT @PoliticaClarin: Rechazan la unificación de dos causas que podría retrasar el primer juicio oral a Cristina Kirchner https://t.co/HJnQSE…,0,1,Kirchner,RT @PoliticaClarin: Rechazan la unificación de dos causas que podría retrasar el primer juicio oral a Cristina Kirchner,RT @PoliticaClarin: Rechazan la unificacion de dos causas que podria retrasar el primer juicio oral a Cristina Kirchner
3,1113516898438848513,2019-04-03 19:01:31,migueldelpalmar,112,"RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…",0,1127,Kirchner,"RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…","RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…"
4,1113516858513268737,2019-04-03 19:01:21,Dgaceyguillo,101,"RT @arilijalad: “Acá hay una forma de apriete. Te piden tirar mierda para arriba. Declará como arrepentido y te vas a tu casa mañana”, cont…",0,73,Kirchner,"RT @arilijalad: “Acá hay una forma de apriete. Te piden tirar mierda para arriba. Declará como arrepentido y te vas a tu casa mañana”, cont…","RT @arilijalad: “Aca hay una forma de apriete. Te piden tirar mierda para arriba. Declara como arrepentido y te vas a tu casa mañana”, cont…"


In [247]:
df['full_text_no_url_no_acc_no_rn'] = df['full_text_no_url_no_acc'].str.replace('\n+|\r+', ' ', case=False)

In [248]:
df.head(5)

,id,created_at,name,followers_count,full_text,favorite_count,retweet_count,flag,full_text_no_url,full_text_no_url_no_acc,full_text_no_url_no_acc_no_rn
0,1113517000167448581,2019-04-03 19:01:55,Dgaceyguillo,101,RT @minutounocom: La joven debíaregresar este jueves a la Argentina tras someterse a un tratamiento médico https://t.co/kWL1dwqCKT,0,13,Kirchner,RT @minutounocom: La joven debíaregresar este jueves a la Argentina tras someterse a un tratamiento médico,RT @minutounocom: La joven debiaregresar este jueves a la Argentina tras someterse a un tratamiento medico,RT @minutounocom: La joven debiaregresar este jueves a la Argentina tras someterse a un tratamiento medico
1,1113516983679619073,2019-04-03 19:01:51,Yolanda22362688,307,"RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner esté acá..."" La verdad que dan asco todos!! Mariano Cúneo…",0,49,Kirchner,"RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner esté acá..."" La verdad que dan asco todos!! Mariano Cúneo…","RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner este aca..."" La verdad que dan asco todos!! Mariano Cuneo…","RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner este aca..."" La verdad que dan asco todos!! Mariano Cuneo…"
2,1113516955158351878,2019-04-03 19:01:44,BramanteLidia,409,RT @PoliticaClarin: Rechazan la unificación de dos causas que podría retrasar el primer juicio oral a Cristina Kirchner https://t.co/HJnQSE…,0,1,Kirchner,RT @PoliticaClarin: Rechazan la unificación de dos causas que podría retrasar el primer juicio oral a Cristina Kirchner,RT @PoliticaClarin: Rechazan la unificacion de dos causas que podria retrasar el primer juicio oral a Cristina Kirchner,RT @PoliticaClarin: Rechazan la unificacion de dos causas que podria retrasar el primer juicio oral a Cristina Kirchner
3,1113516898438848513,2019-04-03 19:01:31,migueldelpalmar,112,"RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…",0,1127,Kirchner,"RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…","RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…","RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…"
4,1113516858513268737,2019-04-03 19:01:21,Dgaceyguillo,101,"RT @arilijalad: “Acá hay una forma de apriete. Te piden tirar mierda para arriba. Declará como arrepentido y te vas a tu casa mañana”, cont…",0,73,Kirchner,"RT @arilijalad: “Acá hay una forma de apriete. Te piden tirar mierda para arriba. Declará como arrepentido y te vas a tu casa mañana”, cont…","RT @arilijalad: “Aca hay una forma de apriete. Te piden tirar mierda para arriba. Declara como arrepentido y te vas a tu casa mañana”, cont…","RT @arilijalad: “Aca hay una forma de apriete. Te piden tirar mierda para arriba. Declara como arrepentido y te vas a tu casa mañana”, cont…"


In [249]:

df['label'] = df['full_text_no_url_no_acc_no_rn'].apply(analisis_watson)

In [250]:
df.label.value_counts()

neutral     825
positive    659
negative    312
error       4  
Name: label, dtype: int64

In [251]:
def watson_error(text):
    try:
      response = nlu.analyze(text=text,features=Features( sentiment=SentimentOptions()), language='es')
    except Exception as e:
        print (e)
    return response.result['sentiment']['document']['label']

In [253]:
df[df['label']=='error']

,id,created_at,name,followers_count,full_text,favorite_count,retweet_count,flag,full_text_no_url,full_text_no_url_no_acc,full_text_no_url_no_acc_no_rn,label
185,1113511470636118016,2019-04-03 18:39:57,PerazziniElena,1386,https://t.co/HbQZAq4sS9,0,0,Kirchner,,,,error
200,1113510969387421696,2019-04-03 18:37:57,LaBandaDiario,796,https://t.co/L3nN0QEFjz https://t.co/L3nN0QEFjz,0,0,Kirchner,,,,error
499,1113501836336672773,2019-04-03 18:01:40,anichechu,209,https://t.co/h0NbTkyvBd,0,0,Kirchner,,,,error
362,1113503256930439168,2019-04-03 18:07:18,msnarg,46965,https://t.co/AEX7QBmoxy,0,0,Lavagna,,,,error


In [255]:
df.head(5)

,id,created_at,name,followers_count,full_text,favorite_count,retweet_count,flag,full_text_no_url,full_text_no_url_no_acc,full_text_no_url_no_acc_no_rn,label
0,1113517000167448581,2019-04-03 19:01:55,Dgaceyguillo,101,RT @minutounocom: La joven debíaregresar este jueves a la Argentina tras someterse a un tratamiento médico https://t.co/kWL1dwqCKT,0,13,Kirchner,RT @minutounocom: La joven debíaregresar este jueves a la Argentina tras someterse a un tratamiento médico,RT @minutounocom: La joven debiaregresar este jueves a la Argentina tras someterse a un tratamiento medico,RT @minutounocom: La joven debiaregresar este jueves a la Argentina tras someterse a un tratamiento medico,neutral
1,1113516983679619073,2019-04-03 19:01:51,Yolanda22362688,307,"RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner esté acá..."" La verdad que dan asco todos!! Mariano Cúneo…",0,49,Kirchner,"RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner esté acá..."" La verdad que dan asco todos!! Mariano Cúneo…","RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner este aca..."" La verdad que dan asco todos!! Mariano Cuneo…","RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner este aca..."" La verdad que dan asco todos!! Mariano Cuneo…",neutral
2,1113516955158351878,2019-04-03 19:01:44,BramanteLidia,409,RT @PoliticaClarin: Rechazan la unificación de dos causas que podría retrasar el primer juicio oral a Cristina Kirchner https://t.co/HJnQSE…,0,1,Kirchner,RT @PoliticaClarin: Rechazan la unificación de dos causas que podría retrasar el primer juicio oral a Cristina Kirchner,RT @PoliticaClarin: Rechazan la unificacion de dos causas que podria retrasar el primer juicio oral a Cristina Kirchner,RT @PoliticaClarin: Rechazan la unificacion de dos causas que podria retrasar el primer juicio oral a Cristina Kirchner,neutral
3,1113516898438848513,2019-04-03 19:01:31,migueldelpalmar,112,"RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…",0,1127,Kirchner,"RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…","RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…","RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…",neutral
4,1113516858513268737,2019-04-03 19:01:21,Dgaceyguillo,101,"RT @arilijalad: “Acá hay una forma de apriete. Te piden tirar mierda para arriba. Declará como arrepentido y te vas a tu casa mañana”, cont…",0,73,Kirchner,"RT @arilijalad: “Acá hay una forma de apriete. Te piden tirar mierda para arriba. Declará como arrepentido y te vas a tu casa mañana”, cont…","RT @arilijalad: “Aca hay una forma de apriete. Te piden tirar mierda para arriba. Declara como arrepentido y te vas a tu casa mañana”, cont…","RT @arilijalad: “Aca hay una forma de apriete. Te piden tirar mierda para arriba. Declara como arrepentido y te vas a tu casa mañana”, cont…",neutral


In [256]:
df.columns

Index(['id', 'created_at', 'name', 'followers_count', 'full_text',
       'favorite_count', 'retweet_count', 'flag', 'full_text_no_url',
       'full_text_no_url_no_acc', 'full_text_no_url_no_acc_no_rn', 'label'],
      dtype='object')

In [257]:
df_final = df[['id', 'created_at', 'name', 'followers_count', 'favorite_count', 'retweet_count', 'flag', 'full_text_no_url_no_acc_no_rn', 'label']]

In [259]:
df_final.head(5)

,id,created_at,name,followers_count,favorite_count,retweet_count,flag,full_text_no_url_no_acc_no_rn,label
0,1113517000167448581,2019-04-03 19:01:55,Dgaceyguillo,101,0,13,Kirchner,RT @minutounocom: La joven debiaregresar este jueves a la Argentina tras someterse a un tratamiento medico,neutral
1,1113516983679619073,2019-04-03 19:01:51,Yolanda22362688,307,0,49,Kirchner,"RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner este aca..."" La verdad que dan asco todos!! Mariano Cuneo…",neutral
2,1113516955158351878,2019-04-03 19:01:44,BramanteLidia,409,0,1,Kirchner,RT @PoliticaClarin: Rechazan la unificacion de dos causas que podria retrasar el primer juicio oral a Cristina Kirchner,neutral
3,1113516898438848513,2019-04-03 19:01:31,migueldelpalmar,112,0,1127,Kirchner,"RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…",neutral
4,1113516858513268737,2019-04-03 19:01:21,Dgaceyguillo,101,0,73,Kirchner,"RT @arilijalad: “Aca hay una forma de apriete. Te piden tirar mierda para arriba. Declara como arrepentido y te vas a tu casa mañana”, cont…",neutral


In [262]:
df_final.rename(columns={"full_text_no_url_no_acc_no_rn": "full_text"},inplace=True)

C:\Users\Dicsys\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [263]:
df_final.head()

,id,created_at,name,followers_count,favorite_count,retweet_count,flag,full_text,label
0,1113517000167448581,2019-04-03 19:01:55,Dgaceyguillo,101,0,13,Kirchner,RT @minutounocom: La joven debiaregresar este jueves a la Argentina tras someterse a un tratamiento medico,neutral
1,1113516983679619073,2019-04-03 19:01:51,Yolanda22362688,307,0,49,Kirchner,"RT @krauseirishotm1: Romina Manguel: ""no hay necesidad de que Florencia Kirchner este aca..."" La verdad que dan asco todos!! Mariano Cuneo…",neutral
2,1113516955158351878,2019-04-03 19:01:44,BramanteLidia,409,0,1,Kirchner,RT @PoliticaClarin: Rechazan la unificacion de dos causas que podria retrasar el primer juicio oral a Cristina Kirchner,neutral
3,1113516898438848513,2019-04-03 19:01:31,migueldelpalmar,112,0,1127,Kirchner,"RT @PissanoV: A ver gente, ustedes tienen idea de la gravedad que tiene el hecho de que Julio De Vido se presente en las listas de diputado…",neutral
4,1113516858513268737,2019-04-03 19:01:21,Dgaceyguillo,101,0,73,Kirchner,"RT @arilijalad: “Aca hay una forma de apriete. Te piden tirar mierda para arriba. Declara como arrepentido y te vas a tu casa mañana”, cont…",neutral


In [264]:
df_final.label.value_counts()

neutral     825
positive    659
negative    312
error       4  
Name: label, dtype: int64

In [266]:
df_final.to_csv(results_csv_path, sep='|')